In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Constant

In [ ]:
API_KEY = '902635857ca292a7bc2c9a8ac5aa773b'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

# Weather Data

In [ ]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return {
        'city': data.get('name'),
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'pressure': data['main']['pressure'],
        'wind_gust_speed': data['wind']['speed']
    }

# read historical data

In [ ]:
def read_historical_data(filename):
    df = pd.read_csv(filename)
    df = df.dropna()
    df = df.drop_duplicates()
    return df

# prepare data for ann

In [ ]:
def prepare_data_for_ann(data):
    features = ['Temp', 'Humidity', 'WindGustSpeed', 'Pressure']
    X = data[features].values
    y = data['Temp'].values 

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y, scaler

# build ANN model

In [ ]:
def build_ann_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# train model

In [ ]:
def train_ann_model(X, y):
    model = build_ann_model(X.shape[1])
    model.fit(X, y, epochs=50, batch_size=32)
    return model

# predict model

In [ ]:
def predict_future_temp(model, current_data):
    prediction = model.predict(current_data)
    return prediction

# weather analysis function

In [ ]:
def weather_view():
    city = "kolkata"
    current_weather = get_current_weather(city)
    historical_data = read_historical_data("/kaggle/input/weather/weather.csv")

    X, y, scaler = prepare_data_for_ann(historical_data)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    ann_model = train_ann_model(X_train, y_train)

    current_data = np.array([[current_weather['current_temp'], 
                              current_weather['humidity'], 
                              current_weather['wind_gust_speed'], 
                              current_weather['pressure']]])
    current_data_scaled = scaler.transform(current_data)

    future_temp = predict_future_temp(ann_model, current_data_scaled)

    print(f"City: {city}, {current_weather['country']}")
    print(f"Current Temp: {current_weather['current_temp']}")
    print(f"Minimum Temperature: {current_weather['temp_min']}°C")
    print(f"Maximum Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Weather Prediction: {current_weather['description']}")
    print(f"Predicted Future Temperature: {round(future_temp[0][0], 2)}°C")

weather_view()